In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F    


## 滑动平均idea 


In [32]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
print(f"a \n {a}")
b = torch.randint(0,10,(3,2)).float() # 0 到10,生成一个size为(3,2)的tensor
print(f"b, \n {b}")
c = a @ b 
print(f"c,\n {c}")

# 第一行2 是1 0 0 和 2 6 6 ， 7 4 5 的dotproduct 
# 第二行8 11 是1 1 0 和 2 6 6 , 7 4 5 的dotproduct
# 第三航14 16 是1 1 0 和 2 6 6 ， 7 4 5 的dotproduct
# 同理，我们可以认为 第一次加了1 个 第二次加了两个，第三次加了3个，那么这个数量和每个row的1的数量相等，我们可以直接做一个normalization
# 
d = c / torch.sum(a,1,keepdim=True)
print(F"d \n {d}")

a 
 tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b, 
 tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c,
 tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
d 
 tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [40]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time , channel 
x = torch.randn(B,T,C)
# 什么是randn 和rand 
x.shape

torch.Size([4, 8, 2])

In [42]:
#we want x[b,t] = mean_{i<=t} x[b,t] 滑动平均
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # 这个输出只有两个维度，C没有取， 为什么
        # previous token are at this batch ndimension, and everything up to and including the t th token, Xprev becomes of shape how many tth element in the path and over C 2dimension
        # print(xprev.shape)
        xbow[b,t] = torch.mean(xprev,0) # 代表在dimension 0上做平均，最后会得到一个C dimensionl ector
        # print(f"xbow's shape is {xbow[b,t].shape}") # it becomes 1 dimensional vector of length C,假如说C是2, 那么维度就是[2] 
        
##### xbow的维度为B T C是一个三维，但是当我们说xbow[b,t]的时候，这也就代表了我们在看xbow[b,t,:] 所有的C都考虑，所以当

In [10]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [43]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

### 解释
-0.0894如何来的？ 是(0.1808+(-0.3596))/2  
0.1490如何来的？ 是((0.1808+(-0.3596))+0.6258)/3

In [33]:
wei = torch.tril(torch.ones(T,T))
print(wei)
wei = wei/wei.sum(1,keepdim=True)
print(wei) # 0代表col 1代表Row
print(f"wei's shape {wei.shape}")

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
wei's shape torch.Size([8, 8])


In [36]:
print(wei.reshape(4,8,8))

RuntimeError: shape '[4, 8, 8]' is invalid for input of size 64

In [34]:
x.shape

torch.Size([4, 8, 2])

xbow2 是这样来的
wei的大小是[8 8] 而 x的大小是[4,8,2] 那么在每次Dot product的时候，每一个wei都会和4个[8,2]的矩阵做运算，也就是说会做四次，得到一个[4,8,2] 所以我们也可以理解为 ，wei被“广播” 但实际上不是广播 成了[4,8,8] 然后每次是[i,8,8]和[i,8,2]在做运算， 这样更高效

In [21]:
xbow2 = wei@x

In [38]:
xbow.shape

torch.Size([4, 8, 2])

In [47]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [48]:
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [49]:
torch.allclose(xbow,xbow2)

False

In [51]:
# 第三种方法 用softmax 
tril = torch.tril(torch.ones(T,T))
wei3 = torch.zeros((T,T))
wei3 = wei3.masked_fill(tril == 0, float('-inf'))
print(wei3)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [52]:
wei3 = F.softmax(wei3,dim=-1)
print(wei3)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [54]:
xbow3 = wei3@x 
print(xbow3)
torch.allclose(xbow3,xbow2)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


True

### A case for one head attention 


In [57]:
# self attention 
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels 
x = torch.randn(B,T,C)
# single head 
head_size = 16
key = nn.Linear(C, head_size,bias=False) # 32,16
query = nn.Linear(C,head_size,bias=False)# 32,16

k = key(x) # [4,8,32] [32,16] -> [4,8,16]
q = query(x) # [4,8,16] B T 16
# 在这一步，也就是说一个x会有两个vector k q 到这里这ector没有communication
# all the queries will dot product with all the keys 
# dot_product = q @ k
## 但是4 8 16 @ 4 8 16 没办法做点乘，我们需要做transpose, 同时要保证batch 
wei = q @ k.transpose(-2,-1) # 2个dim -2代表8 -1代表最后一个维度16, 也就是transpose 交换（倒数第二个维度和导读第一个维度） -》 4 16 8 
#B T 16 @ B 16 T -> B T T
# 所以现在已经不再是一个0 0 comment下面的wei

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim=-1)

out =wei@x 
out.shape

torch.Size([4, 8, 32])

In [61]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

最后一行的最后一个是第八个token, 他知道他的position和他的context 

In [58]:
out

tensor([[[ 1.8077e-01, -6.9988e-02, -3.5962e-01,  ..., -8.0164e-01,
           1.5236e+00,  2.5086e+00],
         [-5.3029e-01, -2.2274e-01,  7.9457e-01,  ...,  1.1658e+00,
           1.5962e+00,  5.5032e-02],
         [-5.9427e-01,  3.1861e-01,  5.8950e-02,  ..., -1.8886e-01,
          -4.4261e-02,  2.0092e-01],
         ...,
         [-1.6099e+00,  1.1770e+00,  1.1506e+00,  ..., -4.9985e-02,
           1.5179e+00, -5.6034e-01],
         [-7.3428e-01,  4.8980e-02,  5.2520e-01,  ...,  8.5586e-01,
           1.0967e+00,  3.4021e-01],
         [-7.5541e-01, -3.2941e-01,  5.4568e-01,  ..., -9.1224e-02,
          -1.4756e-01, -1.3229e-01]],

        [[ 4.5618e-01, -1.0917e+00, -8.2073e-01,  ...,  5.1187e-02,
          -6.5764e-01, -2.5729e+00],
         [ 9.4423e-02,  6.5213e-01, -1.1769e+00,  ...,  6.6192e-01,
          -1.0670e+00,  6.5763e-01],
         [ 1.6564e+00, -3.7253e-01,  1.1281e-01,  ..., -3.9861e-01,
           5.3721e-01, -1.1302e+00],
         ...,
         [ 3.3062e-01,  8

self attention solved this problem :
1. every token will emit two vector: query and key 
2. query : what i m looking for 
3. key : what do i contain
4. we do dot product between query and keys to get the affinity of the token , so my query dot product with all the keys of all the other tokens, it becomes wei here as we defined 
5. if query and key are aligned, they will interact to a very high amount ,高值代表着更有关系

In [64]:
# self attention 
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels 
x = torch.randn(B,T,C)
# single head 
head_size = 16
key = nn.Linear(C, head_size,bias=False) # 32,16
query = nn.Linear(C,head_size,bias=False)# 32,16

k = key(x) # [4,8,32] [32,16] -> [4,8,16]
q = query(x) # [4,8,16] B T 16
# 在这一步，也就是说一个x会有两个vector k q 到这里这ector没有communication
# all the queries will dot product with all the keys 
# dot_product = q @ k
## 但是4 8 16 @ 4 8 16 没办法做点乘，我们需要做transpose, 同时要保证batch 
wei = q @ k.transpose(-2,-1) # 2个dim -2代表8 -1代表最后一个维度16, 也就是transpose 交换（倒数第二个维度和导读第一个维度） -》 4 16 8 
#B T 16 @ B 16 T -> B T T
# 所以现在已经不再是一个0 0 comment下面的wei

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
# wei = wei.masked_fill(tril == 0, float('-inf'))
# wei = F.softmax(wei,dim=-1)

out =wei@x 
wei[0]

tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

在这一步可以看出，每个query和key的dot product,我们这里没有对WEI做任何的mask处理，也就是说每一个query和每一个key做dot product，即使在第一行代表第一个token
但实际上来说，在第一个token，他不应该和后面的token 有什么communication 所以我们用下面的步骤


In [65]:
# self attention 
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels 
x = torch.randn(B,T,C)
# single head 
head_size = 16
key = nn.Linear(C, head_size,bias=False) # 32,16
query = nn.Linear(C,head_size,bias=False)# 32,16

k = key(x) # [4,8,32] [32,16] -> [4,8,16]
q = query(x) # [4,8,16] B T 16
# 在这一步，也就是说一个x会有两个vector k q 到这里这ector没有communication
# all the queries will dot product with all the keys 
# dot_product = q @ k
## 但是4 8 16 @ 4 8 16 没办法做点乘，我们需要做transpose, 同时要保证batch 
wei = q @ k.transpose(-2,-1) # 2个dim -2代表8 -1代表最后一个维度16, 也就是transpose 交换（倒数第二个维度和导读第一个维度） -》 4 16 8 
#B T 16 @ B 16 T -> B T T
# 所以现在已经不再是一个0 0 comment下面的wei

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
# wei = F.softmax(wei,dim=-1)

out =wei@x 
wei[0]

tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

这样 我们就可以通过tril这个triangular 来限制一些communication ，但是有-inf和负数，我们不好做，我们需要一个正常的distribution所以我们exponentiate 并 normalize


In [66]:
# self attention 
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels 
x = torch.randn(B,T,C)
# single head 
head_size = 16
key = nn.Linear(C, head_size,bias=False) # 32,16
query = nn.Linear(C,head_size,bias=False)# 32,16

k = key(x) # [4,8,32] [32,16] -> [4,8,16]
q = query(x) # [4,8,16] B T 16
# 在这一步，也就是说一个x会有两个vector k q 到这里这ector没有communication
# all the queries will dot product with all the keys 
# dot_product = q @ k
## 但是4 8 16 @ 4 8 16 没办法做点乘，我们需要做transpose, 同时要保证batch 
wei = q @ k.transpose(-2,-1) # 2个dim -2代表8 -1代表最后一个维度16, 也就是transpose 交换（倒数第二个维度和导读第一个维度） -》 4 16 8 
#B T 16 @ B 16 T -> B T T
# 所以现在已经不再是一个0 0 comment下面的wei

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim=-1)

out =wei@x 
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
但实际上在我们计算attention的时候，我们还有一个V value,sssSsasd